# Find Diverse set based on Embeddings 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix
import pandas as pd    
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Creating the input data

In [ ]:
df = pd.read_csv(r'/content/drive/MyDrive/SemSeg2/data/leftImg8bit_embed_EXP_2.csv')  # SAME EMBEDDING ALL SAMPLE SLICES IN EXP 1 
df = df.iloc[: , 1:]
df.head(10)

,RawImage,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.009482,0.040444,0.176667,-0.378674,0.319166,0.155639,-0.133415,-0.072951,0.021591,0.031846,-0.110247,-0.272506,0.057182,-0.224349,0.042039,0.004738,0.057219,0.078108,-0.137147,0.192808,0.325018,-0.086795,-0.060851,0.157772,-0.258219,0.141029,0.125618,-0.164265,-0.005470,-0.308155,0.234430,-0.211591
1,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,-0.057985,0.032342,0.228185,-0.331204,0.300958,0.137861,-0.107074,-0.030073,0.060301,0.088352,-0.050845,-0.255311,-0.024984,-0.268264,0.098715,-0.065581,0.025577,0.106971,-0.146270,0.246646,0.251633,-0.097930,-0.174280,0.125117,-0.242944,0.211897,0.093642,-0.196171,-0.042080,-0.358664,0.182824,-0.133256
2,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,-0.035203,0.035616,0.213378,-0.339761,0.310516,0.135883,-0.119975,-0.042616,0.046704,0.075060,-0.060332,-0.266416,-0.007096,-0.264107,0.083883,-0.045261,0.031547,0.103629,-0.144863,0.228679,0.274687,-0.099647,-0.143618,0.126185,-0.247702,0.198394,0.098999,-0.192216,-0.035667,-0.354396,0.197263,-0.154658
3,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.054608,-0.009370,0.194284,-0.365240,0.345106,0.154503,-0.140499,-0.080542,0.011912,0.040552,-0.123583,-0.245331,0.046315,-0.144162,-0.009100,0.022404,0.077584,0.112068,-0.146053,0.266314,0.321284,-0.054181,-0.099733,0.233516,-0.237760,0.105481,0.106370,-0.154497,0.024998,-0.311352,0.205571,-0.178032
4,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,-0.063562,-0.023459,0.266483,-0.253898,0.288249,0.109860,-0.095287,-0.004078,0.080825,0.131696,-0.005988,-0.202262,-0.114567,-0.219800,0.097409,-0.105440,0.021682,0.178489,-0.156425,0.341583,0.157499,-0.067340,-0.304672,0.143087,-0.189436,0.239976,0.035331,-0.206173,-0.045083,-0.379823,0.095734,-0.001813
5,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,-0.031710,-0.053550,0.271587,-0.230415,0.295743,0.088854,-0.108158,-0.011996,0.068302,0.141652,-0.010353,-0.193376,-0.122527,-0.167510,0.069130,-0.092416,0.027720,0.206209,-0.152922,0.377134,0.151081,-0.049857,-0.320466,0.171441,-0.169987,0.223471,0.018243,-0.195479,-0.013879,-0.391751,0.074147,0.024600
6,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.061313,-0.107834,0.249771,-0.248180,0.344102,0.106450,-0.134253,-0.037033,0.023670,0.105561,-0.064624,-0.169948,-0.069667,-0.034189,-0.028191,-0.018186,0.057749,0.221002,-0.153349,0.425803,0.205416,-0.017189,-0.281558,0.283443,-0.146601,0.127006,0.022851,-0.159184,0.045533,-0.360519,0.070342,0.002129
7,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.023745,0.007415,0.201290,-0.361108,0.331438,0.148015,-0.139026,-0.069563,0.021865,0.049637,-0.107110,-0.260643,0.030287,-0.185585,0.024500,-0.001942,0.062958,0.112062,-0.143378,0.257703,0.309238,-0.070405,-0.113095,0.193641,-0.244690,0.138930,0.104048,-0.169825,0.009100,-0.331678,0.204164,-0.171018
8,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.091412,-0.111418,0.236448,-0.263886,0.355639,0.114001,-0.135579,-0.052413,0.010141,0.090003,-0.083086,-0.167394,-0.042662,-0.004997,-0.057527,0.002623,0.072106,0.210710,-0.156003,0.420795,0.228935,-0.005551,-0.252069,0.311604,-0.148259,0.094713,0.033715,-0.147449,0.061709,-0.340370,0.082723,-0.019515
9,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,-0.049315,0.031944,0.228043,-0.332675,0.304090,0.140206,-0.110635,-0.031085,0.057678,0.085592,-0.051433,-0.254277,-0.021826,-0.268503,0.097098,-0.063599,0.028789,0.107832,-0.149181,0.245507,0.254228,-0.097666,-0.169796,0.121434,-0.244114,0.210506,0.090270,-0.198508,-0.043426,-0.357351,0.183142,-0.131228


In [ ]:
input_features=df.iloc[: , 1:]
input_features.shape

(2975, 32)

# SET SAMPLE SIZE

In [ ]:
#Select the total clusters , also the number of samples we want at the end 
S=100    #also change depending on sample slice   !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# CLUSTERING 

In [ ]:
#Set Up data sets to be referencing the same random set Cx and Number of randomly chosen clusers S as is applied to random. 
Cx=input_features
#total number clusters is the number of samples we want.
n_clusters=S

#Use best Cluster method that searches sample space , and finds groupings(clusters) that are similar
#The Agglomerative method will stop when the sample size has been reached
#Euclidian distance is used to find most similar groupings and put into the same cluster
#Ward linkage method is used to account of extremities
#Each point in random set is eventually given a cluster , and total number clusters is the number of samples we want.  
from sklearn.cluster import AgglomerativeClustering
cluster = AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean', linkage='ward')  
k=cluster.fit_predict(Cx)
Data=pd.DataFrame(Cx)
Cluster=pd.DataFrame(k , columns=['Cluster'])
result = pd.concat([Data, Cluster], axis=1)

#Select Nearest Center

In [ ]:
#Many records per cluster , need to find the best distanced points 
#result['Cluster'].value_counts().sort_index(ascending=False).plot(kind="barh" , figsize=(30, 22))

In [ ]:
#Find centroid of each cluster 
CenterMatrix=result.groupby("Cluster").agg({'0' : 'mean'  , '1' : 'mean', '2' : 'mean' , '3' : 'mean', '4' : 'mean' , '5' : 'mean', '6' : 'mean' , '7' : 'mean', '8' : 'mean' ,'9' : 'mean', '10' : 'mean' , '11' : 'mean', '12' : 'mean' , '13' : 'mean', '14' : 'mean' , '15' : 'mean', '16' : 'mean' ,'17' : 'mean', '18' : 'mean' , '19' : 'mean', '20' : 'mean' , '21' : 'mean', '22' : 'mean' , '23' : 'mean', '24' : 'mean' ,'25' : 'mean', '26' : 'mean' , '27' : 'mean', '28' : 'mean' , '29' : 'mean', '30' : 'mean' , '31' : 'mean'        })


In [ ]:
#Make a center cluster for each of the image rows
H=pd.merge(result, CenterMatrix, how='inner', left_on='Cluster', right_on='Cluster')

In [ ]:
#Compute each points distance from its center 


#importing numpy
import numpy as np

#initializing two arrays
array1 = np.array(H.iloc[ : ,33:])
array2 = np.array(result.iloc[0 , :-1 ])

#computing the Euclidan distance 
temp = array1 - array2
distance = np.sqrt(np.sum(np.square(temp)))


print("Euclidean Distance: ", distance)


Euclidean Distance:  48.813277850454156


In [ ]:
#initializing two arrays
ls=list()
for i in range(len(H)):
  array1 = np.array(H.iloc[ i:i+1 ,33:]) #center points
  array2 = np.array(result.iloc[i , :-1 ]) #Matrix points
  temp = array1 - array2
  distance = np.sqrt(np.sum(np.square(temp)))
  ls.append([i,distance])

euclid = pd.DataFrame(ls,columns=["rec","distance"])

In [ ]:
final = pd.concat([result, euclid], axis=1)


In [ ]:
# Select for sample the true point that gives us the minimum distance 
out2=final.loc[final.groupby('Cluster').distance.idxmin()] #true points 

In [ ]:
out2.shape

(100, 35)

In [ ]:
out2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,Cluster,rec,distance
623,-0.036130,0.053684,0.198261,-0.358689,0.305175,0.150682,-0.114318,-0.046361,0.043728,0.060158,-0.075988,-0.270469,0.021611,-0.273148,0.085070,-0.036438,0.036148,0.079569,-0.141429,0.197053,0.292156,-0.102647,-0.104213,0.118477,-0.260952,0.181071,0.117391,-0.184912,-0.035182,-0.331769,0.215859,-0.186212,0,623,0.027832
1625,-0.056989,-0.105374,0.287223,-0.070473,0.222779,0.053616,-0.035454,0.046638,0.113392,0.174124,0.062598,-0.044199,-0.212740,-0.076310,0.063371,-0.158363,0.022816,0.241683,-0.146041,0.430275,-0.025527,0.009582,-0.449838,0.161990,-0.064171,0.217128,-0.054998,-0.167467,-0.042807,-0.308557,-0.062750,0.187025,1,1625,0.160014
2161,0.039353,-0.172279,0.257849,-0.046107,0.257241,0.045110,-0.051955,0.025960,0.064115,0.154720,0.026369,-0.003630,-0.187005,0.076887,-0.043016,-0.090930,0.052664,0.274713,-0.142380,0.474844,-0.018506,0.063650,-0.436579,0.271190,-0.008733,0.127152,-0.064414,-0.116989,0.032370,-0.270358,-0.083189,0.202853,2,2161,0.030365
2728,-0.054782,-0.144625,0.237887,0.093894,0.111663,-0.017534,0.002271,0.075718,0.103463,0.173460,0.124799,0.063587,-0.260352,0.025382,0.044679,-0.169423,0.000674,0.248950,-0.099643,0.389989,-0.181183,0.053309,-0.473244,0.109674,0.051705,0.175327,-0.117920,-0.113265,-0.038020,-0.219786,-0.170767,0.312904,3,2728,0.039915
890,0.002500,-0.018530,0.235542,-0.320033,0.324601,0.139166,-0.127946,-0.042228,0.043363,0.084807,-0.070471,-0.244289,-0.028742,-0.189223,0.049336,-0.032078,0.049493,0.144397,-0.149636,0.315360,0.262220,-0.071107,-0.197175,0.191312,-0.223905,0.177589,0.075076,-0.186496,-0.001179,-0.364970,0.157591,-0.102415,4,890,0.150522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1571,-0.004153,-0.135997,0.286664,-0.093739,0.258140,0.057383,-0.064588,0.032299,0.077715,0.167146,0.031695,-0.058448,-0.190683,-0.024847,0.015880,-0.113582,0.039588,0.256519,-0.147537,0.457471,0.013268,0.020926,-0.432684,0.216681,-0.064167,0.176399,-0.041843,-0.154271,-0.002356,-0.320362,-0.050181,0.166480,95,1571,0.149358
2526,-0.054963,-0.120939,0.278153,-0.027983,0.187828,0.025067,-0.033172,0.056204,0.102888,0.184805,0.081887,-0.017642,-0.233320,-0.046631,0.059958,-0.160541,0.011313,0.246639,-0.137193,0.426969,-0.069278,0.010976,-0.457357,0.145183,-0.033587,0.208669,-0.079490,-0.158760,-0.038324,-0.300355,-0.099360,0.226478,96,2526,0.188975
179,0.096975,-0.124032,0.237474,-0.248730,0.358852,0.118065,-0.121289,-0.044014,0.012828,0.094471,-0.084340,-0.141413,-0.047574,0.024836,-0.073233,0.003382,0.077668,0.221177,-0.159150,0.430435,0.213146,0.016374,-0.270046,0.334403,-0.132244,0.086862,0.028989,-0.135853,0.071279,-0.320000,0.072374,-0.009002,97,179,0.062936
1979,-0.067976,-0.122377,0.264343,0.032258,0.138748,0.013907,-0.000131,0.071506,0.110586,0.180895,0.105477,0.029817,-0.252956,-0.022409,0.061667,-0.180236,0.008112,0.239336,-0.124860,0.406183,-0.131696,0.035824,-0.475199,0.117641,0.006549,0.201949,-0.103756,-0.138744,-0.052032,-0.248391,-0.143151,0.274302,98,1979,0.129526


In [ ]:
#Link it to name of files on index 

selected=pd.merge(df, out2, left_index=True, right_index=True)
selected

,RawImage,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,12_x,13_x,14_x,15_x,16_x,17_x,18_x,19_x,20_x,21_x,22_x,23_x,24_x,25_x,26_x,27_x,28_x,29_x,30_x,31_x,0_y,1_y,2_y,3_y,4_y,5_y,6_y,7_y,8_y,9_y,10_y,11_y,12_y,13_y,14_y,15_y,16_y,17_y,18_y,19_y,20_y,21_y,22_y,23_y,24_y,25_y,26_y,27_y,28_y,29_y,30_y,31_y,Cluster,rec,distance
0,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.009482,0.040444,0.176667,-0.378674,0.319166,0.155639,-0.133415,-0.072951,0.021591,0.031846,-0.110247,-0.272506,0.057182,-0.224349,0.042039,0.004738,0.057219,0.078108,-0.137147,0.192808,0.325018,-0.086795,-0.060851,0.157772,-0.258219,0.141029,0.125618,-0.164265,-0.005470,-0.308155,0.234430,-0.211591,0.009482,0.040444,0.176667,-0.378674,0.319166,0.155639,-0.133415,-0.072951,0.021591,0.031846,-0.110247,-0.272506,0.057182,-0.224349,0.042039,0.004738,0.057219,0.078108,-0.137147,0.192808,0.325018,-0.086795,-0.060851,0.157772,-0.258219,0.141029,0.125618,-0.164265,-0.005470,-0.308155,0.234430,-0.211591,47,0,0.012371
3,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.054608,-0.009370,0.194284,-0.365240,0.345106,0.154503,-0.140499,-0.080542,0.011912,0.040552,-0.123583,-0.245331,0.046315,-0.144162,-0.009100,0.022404,0.077584,0.112068,-0.146053,0.266314,0.321284,-0.054181,-0.099733,0.233516,-0.237760,0.105481,0.106370,-0.154497,0.024998,-0.311352,0.205571,-0.178032,0.054608,-0.009370,0.194284,-0.365240,0.345106,0.154503,-0.140499,-0.080542,0.011912,0.040552,-0.123583,-0.245331,0.046315,-0.144162,-0.009100,0.022404,0.077584,0.112068,-0.146053,0.266314,0.321284,-0.054181,-0.099733,0.233516,-0.237760,0.105481,0.106370,-0.154497,0.024998,-0.311352,0.205571,-0.178032,78,3,0.186797
7,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.023745,0.007415,0.201290,-0.361108,0.331438,0.148015,-0.139026,-0.069563,0.021865,0.049637,-0.107110,-0.260643,0.030287,-0.185585,0.024500,-0.001942,0.062958,0.112062,-0.143378,0.257703,0.309238,-0.070405,-0.113095,0.193641,-0.244690,0.138930,0.104048,-0.169825,0.009100,-0.331678,0.204164,-0.171018,0.023745,0.007415,0.201290,-0.361108,0.331438,0.148015,-0.139026,-0.069563,0.021865,0.049637,-0.107110,-0.260643,0.030287,-0.185585,0.024500,-0.001942,0.062958,0.112062,-0.143378,0.257703,0.309238,-0.070405,-0.113095,0.193641,-0.244690,0.138930,0.104048,-0.169825,0.009100,-0.331678,0.204164,-0.171018,28,7,0.131037
20,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.009451,0.023794,0.199957,-0.360326,0.326306,0.143805,-0.136230,-0.061604,0.024411,0.052079,-0.093591,-0.268067,0.031918,-0.222269,0.043867,-0.008585,0.051588,0.101792,-0.141111,0.226405,0.310120,-0.085568,-0.100073,0.163853,-0.247180,0.155269,0.110127,-0.174697,-0.003538,-0.334907,0.213313,-0.183267,0.009451,0.023794,0.199957,-0.360326,0.326306,0.143805,-0.136230,-0.061604,0.024411,0.052079,-0.093591,-0.268067,0.031918,-0.222269,0.043867,-0.008585,0.051588,0.101792,-0.141111,0.226405,0.310120,-0.085568,-0.100073,0.163853,-0.247180,0.155269,0.110127,-0.174697,-0.003538,-0.334907,0.213313,-0.183267,39,20,0.085956
22,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,-0.032185,0.023353,0.230322,-0.331945,0.321241,0.136211,-0.113956,-0.035187,0.050746,0.090060,-0.057973,-0.257225,-0.016839,-0.247958,0.077575,-0.052467,0.040693,0.116111,-0.147525,0.251081,0.266992,-0.088809,-0.168049,0.141275,-0.237922,0.192341,0.094069,-0.193995,-0.030753,-0.357907,0.183195,-0.135865,-0.032185,0.023353,0.230322,-0.331945,0.321241,0.136211,-0.113956,-0.035187,0.050746,0.090060,-0.057973,-0.257225,-0.016839,-0.247958,0.077575,-0.052467,0.040693,0.116111,-0.147525,0.251081,0.266992,-0.088809,-0.168049,0.141275,-0.237922,0.192341,0.094069,-0.193995,-0.030753,-0.357907,0.183195,-0.135865,49,22,0.058218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2775,/content/drive/MyDrive/S

#Record Metric

In [ ]:
#Total Distance and average distance for this set 

print(out2['distance'].sum())
print(out2['distance'].sum()/S)


11.860975046871413
0.11860975046871414



N100:11.860975046871413
0.11860975046871414

n200 30.133124612329254
0.15066562306164627

n500:119.67329636933907
0.23934659273867814


N1000 ; 340.28352766376395
0.34028352766376396

N1500 : 638.1755415131543
0.4254503610087695

N2000 : 940.8594743722952
0.47042973718614756

N2500 : 1139.29486706829
0.455717946827316

# CREATE FINAL DATAFRAME 

In [ ]:
#fetch masks file names 
K=pd.DataFrame(selected['RawImage'])
K


,RawImage
0,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
3,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
7,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
20,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
22,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
...,...
2775,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
2786,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
2922,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
2952,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...


In [ ]:
#Fetch Corrisponding Ground Mask From Meta Data 
import os
os.chdir("/content/drive/MyDrive/SemSeg2/Experiments/CityScapes")

m= pd.read_csv(r'CityScapes_MetaData.csv')
m.head(10)

,Unnamed: 0,ImageName,RawImage,Mask,ColourMask
0,0,aachen_000000_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
1,1,aachen_000001_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
2,2,aachen_000002_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
3,3,aachen_000003_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
4,4,aachen_000004_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
5,5,aachen_000005_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
6,6,aachen_000006_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
7,7,aachen_000007_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
8,8,aachen_000008_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
9,9,aachen_000009_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...


In [ ]:
J = pd.merge(K, m, how="inner", on="RawImage")
J[['ImageName' ,	'RawImage','Mask'	, 'ColourMask']]

,ImageName,RawImage,Mask,ColourMask
0,aachen_000000_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
1,aachen_000003_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
2,aachen_000007_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
3,aachen_000020_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
4,aachen_000022_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
...,...,...,...,...
95,weimar_000064_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
96,weimar_000075_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
97,zurich_000069_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
98,zurich_000099_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...


#CHANGE FILE NAME

In [ ]:
#Write to a data frame 

import os
os.chdir("/content/drive/MyDrive/SemSeg2/Experiments/CityScapes/CoreSet_1")


J.to_csv('CityScapes_Diversity_EXP2_100.csv')
#J.to_csv('CityScapes_Diversity_EXP2_200.csv')
#J.to_csv('CityScapes_Diversity_EXP2_500.csv')
#J.to_csv('CityScapes_Diversity_EXP2_1000.csv')
#J.to_csv('CityScapes_Diversity_EXP2_1500.csv')
#J.to_csv('CityScapes_Diversity_EXP2_2000.csv')
#J.to_csv('CityScapes_Diversity_EXP2_2500.csv')

#SHOW VISUALS OF CLUSTERS 

In [ ]:
pd.set_option('display.max_rows', None)
k=result[['Cluster']].value_counts()
k

In [ ]:
selectedX=pd.merge(df, result, left_index=True, right_index=True)
selectedX

In [ ]:
view = selectedX[['RawImage','Cluster']]
v=view.loc[view['Cluster']==60]['RawImage'].tolist()
v

['/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/bremen/bremen_000240_000019_leftImg8bit.png',
 '/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/bremen/bremen_000248_000019_leftImg8bit.png',
 '/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/jena/jena_000050_000019_leftImg8bit.png',
 '/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/zurich/zurich_000050_000019_leftImg8bit.png']

In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# loads the image in color
img = cv2.imread(v[0])
cv2_imshow(img)

In [ ]:
img = cv2.imread(v[1])
cv2_imshow(img)

In [ ]:
img = cv2.imread(v[2])
cv2_imshow(img)

# POPULATE INTO FOLDERS ---> Clear and add to folder 

MAKE SURE ALLIGNMENT BETWEEN THE IMAGES AND MASKS 

In [107]:
#READ EXPERIMENT FILE 
import pandas as pd
import os
os.chdir("/content/drive/MyDrive/SemSeg2/Experiments/CityScapes/CoreSet_1")

#Same core embedding different clustering seed
df = pd.read_csv('CityScapes_Diversity_EXP1_100.csv')  
#df = pd.read_csv('CityScapes_Diversity_EXP1_200.csv')  
#df = pd.read_csv('CityScapes_Diversity_EXP1_500.csv')  
#df = pd.read_csv('CityScapes_Diversity_EXP1_1000.csv')  
#df = pd.read_csv('CityScapes_Diversity_EXP1_1500.csv')  
#df = pd.read_csv('CityScapes_Diversity_EXP1_2000.csv')  
#df = pd.read_csv('CityScapes_Diversity_EXP1_2500.csv')  


#df=J

r = df['RawImage']
ls_mask = df['Mask']


In [108]:
len(df)

100

In [109]:
#clear the images
mydir = '/content/drive/MyDrive/SemSeg2/data_random/leftImg8bit/train/all'

filelist = [ f for f in os.listdir(mydir) ]
for f in filelist:
  os.remove(os.path.join(mydir, f))

#clear masks 
mydir = '/content/drive/MyDrive/SemSeg2/data_random/gtFine/train/all'

filelist = [ f for f in os.listdir(mydir) ]
for f in filelist:
  os.remove(os.path.join(mydir, f))

In [110]:
#Now copy into a directory into the two respoective folders 
#print(r)

import shutil

#clear all files but not the folder stucture 

for i in range(len(r)):
  shutil.copy(r[i], r'/content/drive/MyDrive/SemSeg2/data_random/leftImg8bit/train/all')
  print(r[i])

/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000002_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000007_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000038_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000141_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000170_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/bochum/bochum_000000_017216_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/bochum/bochum_000000_038022_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/bremen/bremen_000017_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/bremen/bremen_000019_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/bremen/bremen_000026_000019_leftImg8bit.png
/content/drive/MyDri

In [111]:
#Now copy into a directory into the two respective folders 
import shutil

#clear all files but not the folder stucture 

for i in range(len(ls_mask)):
  shutil.copy(ls_mask[i], r'/content/drive/MyDrive/SemSeg2/data_random/gtFine/train/all')
  print(ls_mask[i])

/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000002_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000007_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000038_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000141_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000170_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/bochum/bochum_000000_017216_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/bochum/bochum_000000_038022_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/bremen/bremen_000017_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/bremen/bremen_000019_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/bremen/bremen_000026_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2

# Overlap of images in Experiment sets 

In [ ]:
a = pd.read_csv('CityScapes_Diversity_EXP1_1000.csv')  
b = pd.read_csv('CityScapes_Diversity_EXP2_1000.csv')  

list1 = a['ImageName']
list2 = b['ImageName']

intersection_set_ab = set.intersection(set(list1), set(list2))

len(intersection_set_ab)/1000

0.388

In [ ]:
a = pd.read_csv('CityScapes_Diversity_EXP1_1500.csv')  
b = pd.read_csv('CityScapes_Diversity_EXP2_1500.csv')  

list1 = a['ImageName']
list2 = b['ImageName']

intersection_set_ab = set.intersection(set(list1), set(list2))

len(intersection_set_ab)/1500

0.5746666666666667

In [ ]:
a = pd.read_csv('CityScapes_Diversity_EXP1_2000.csv')  
b = pd.read_csv('CityScapes_Diversity_EXP2_2000.csv')  

list1 = a['ImageName']
list2 = b['ImageName']

intersection_set_ab = set.intersection(set(list1), set(list2))

len(intersection_set_ab)/2000

0.728

In [ ]:
a = pd.read_csv('CityScapes_Diversity_EXP1_2500.csv')  #over wrote this by mistake
b = pd.read_csv('CityScapes_Diversity_EXP2_2500.csv')  

list1 = a['ImageName']
list2 = b['ImageName']

intersection_set_ab = set.intersection(set(list1), set(list2))

len(intersection_set_ab)/2500

1.0